In [12]:
import os
# 设置镜像
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [13]:
# 先试用模型 - 简单的文本生成测试
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import torch

# 设置缓存目录
cache_dir = "../models_cache"  # 您可以修改为任意您想要的路径
os.makedirs(cache_dir, exist_ok=True)

# 使用指定的缓存目录下载模型
# TODO 换小点的huggingface模型测试
model = AutoModelForSeq2SeqLM.from_pretrained(
    pretrained_model_name_or_path="bigscience/mt0-small",
    cache_dir=cache_dir
)
# 加载对应的tokenizer
tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-large")

In [14]:
# 测试输入
test_text = "translate English to chinese: Hello, how are you ?"
print(f"输入文本: {test_text}")

# 编码输入
inputs = tokenizer(test_text, return_tensors="pt")
print(f"Token IDs: {inputs['input_ids']}")
print(f"输入长度: {inputs['input_ids'].shape}")

# 生成输出（简单测试）
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'], 
        max_length=50, 
        num_beams=2, 
        early_stopping=True
    )

# 解码输出
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"生成结果: {generated_text}")

输入文本: translate English to chinese: Hello, how are you ?
Token IDs: tensor([[ 37194,   5413,    288,    259, 116983,    267,  30273,    261,   2606,
            418,    521,    259,    291,      1]])
输入长度: torch.Size([1, 14])
生成结果: Bonjour,你呢?
生成结果: Bonjour,你呢?


In [15]:
# 测试输入
test_text = "翻译成英文：你好，你是谁？"
print(f"输入文本: {test_text}")

# 编码输入
inputs = tokenizer(test_text, return_tensors="pt")
print(f"Token IDs: {inputs['input_ids']}")
print(f"输入长度: {inputs['input_ids'].shape}")

# 生成输出（简单测试）
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'], 
        max_length=50, 
        num_beams=2, 
        early_stopping=True
    )

# 解码输出
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"生成结果: {generated_text}")

输入文本: 翻译成英文：你好，你是谁？
Token IDs: tensor([[   259, 138961,   4449, 102031,    267,   4235,   3586,    261,   4235,
         156695,    291,      1]])
输入长度: torch.Size([1, 12])
生成结果: 你好,你好?
生成结果: 你好,你好?


In [16]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [17]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 344,064 || all params: 300,520,832 || trainable%: 0.1145


In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../train_cache/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [19]:
# 加载中文指令数据集
from datasets import load_dataset, Dataset
import pandas as pd

# 方案1: 使用BelleGroup/train_2M_CN (中文指令数据集)
# 这是一个高质量的中文指令微调数据集
try:
    # 加载数据集的一个子集用于演示
    dataset = load_dataset("BelleGroup/train_2M_CN", split="train[:1000]", cache_dir=cache_dir)
    print("成功加载 BelleGroup/train_2M_CN 数据集")
except:
    print("无法加载 BelleGroup/train_2M_CN，尝试其他数据集...")
    
    # 方案2: 使用tatsu-lab/alpaca_chinese (中文Alpaca数据集)
    try:
        dataset = load_dataset("tatsu-lab/alpaca_chinese", split="train[:1000]", cache_dir=cache_dir)
        print("成功加载 tatsu-lab/alpaca_chinese 数据集")
    except:
        print("无法加载 tatsu-lab/alpaca_chinese，创建示例数据集...")
        
        # 方案3: 创建示例中文指令数据集
        sample_data = [
            {
                "instruction": "将下面的句子翻译成英文",
                "input": "今天天气很好。",
                "output": "The weather is very nice today."
            },
            {
                "instruction": "回答下面的问题",
                "input": "什么是人工智能？",
                "output": "人工智能（AI）是指让机器模拟人类智能的技术，包括学习、推理、感知等能力。"
            },
            {
                "instruction": "总结下面的文本",
                "input": "机器学习是人工智能的一个重要分支，它使计算机能够在不被明确编程的情况下学习。",
                "output": "机器学习是AI的分支，让计算机自动学习而无需明确编程。"
            }
        ] * 100  # 重复数据以创建更大的数据集
        
        dataset = Dataset.from_list(sample_data)
        print("创建了示例中文指令数据集")

print(f"数据集大小: {len(dataset)}")
print("数据集示例:")
for i in range(min(3, len(dataset))):
    print(f"示例 {i+1}:")
    print(f"  指令: {dataset[i]['instruction']}")
    print(f"  输入: {dataset[i]['input']}")
    print(f"  输出: {dataset[i]['output']}")
    print()

成功加载 BelleGroup/train_2M_CN 数据集
数据集大小: 1000
数据集示例:
示例 1:
  指令: 对以下代码进行调试。
def add(a,b):
  return a + b
  print(add(2,3))
  输入: 
  输出: 这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是说函数在 `return` 语句处结束，因此 `print` 语句永远不会被执行。
要解决这个问题，只需在 `return` 语句之前移到 `print` 语句。修改后的代码如下所示：
```
def add(a, b):
    print(a + b)
    return a + b
add(2, 3)
```
这样就可以在调用 `add` 函数时打印出结果，并在函数内部返回 `a + b` 值，使其在需要时可以接收和使用。同时，注意在调用 `add` 函数时，不需要再使用 `print` 方法了。

示例 2:
  指令: 为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。

  输入: 
  输出: 为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：
1. 名称应该简单易记，不宜太长；
2. 名称应该与应用程序的主要功能或特点相关；
3. 名称应该引起人们的兴趣和好奇心。
举个例子，如果我们的应用程序是一款健康管理应用程序，那么可以考虑以下几个名称：
1. HealthBoss
2. FitMeUp
3. HealthMate
4. BodyBest
5. HealthLab
其中，HealthBoss强调应用程序可帮助用户成为健康的老板，FitMeUp强调应用程序可帮助用户达到健康生活的目标，HealthMate强调应用程序是用户健康的伙伴，BodyBest强调应用程序可通过健康管理帮助用户获得最佳身体状态，HealthLab强调应用程序可帮助用户进行健康诊断和实验。
一句简短的介绍可包含以下几个要素：
1. 应用程序的名称；
2. 应用程序的主要功能或特点；
3. 应用程序的受众或目标用户。
例如，以上面的例子为基础，可以写出以下几个简短的介绍：
1. HealthBoss – 健康管理，你的健康老板。
2. FitMeUp – 达成健康目标，轻松自在，适合每个人。
3. Hea

In [20]:
# 数据预处理函数
def preprocess_function(examples):
    """
    将指令数据转换为模型输入格式
    """
    inputs = []
    targets = []
    
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        # 构建输入文本：指令 + 输入
        if input_text and input_text.strip():
            input_text_formatted = f"指令: {instruction}\n输入: {input_text}\n输出: "
        else:
            input_text_formatted = f"指令: {instruction}\n输出: "
        
        inputs.append(input_text_formatted)
        targets.append(output)
    
    # Tokenize输入和输出
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)
    labels = tokenizer(targets, max_length=512, truncation=True, padding=True)
    
    # 设置labels
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# 应用预处理
print("开始数据预处理...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# 分割数据集
train_test_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
tokenized_datasets = {
    "train": train_test_split["train"],
    "test": train_test_split["test"]
}

print(f"训练集大小: {len(tokenized_datasets['train'])}")
print(f"测试集大小: {len(tokenized_datasets['test'])}")

# 数据收集器
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

开始数据预处理...
训练集大小: 900
测试集大小: 100


In [21]:
# 评估指标
import numpy as np
import evaluate

def computeMetrics(evalPreds):
    """
    计算评估指标
    """
    predictions, labels = evalPreds
    
    try:
        # 解码预测结果和标签
        if isinstance(predictions, tuple):
            predictions = predictions[0]
        
        # 将-100替换为tokenizer.pad_token_id用于解码
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        
        # 解码
        decodedPreds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decodedLabels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        # 计算BLEU分数（如果可用）
        bleuMetric = evaluate.load("bleu")
        bleuScore = bleuMetric.compute(
            predictions=decodedPreds, 
            references=[[label] for label in decodedLabels]
        )
        
        # 计算平均长度作为辅助指标
        predLens = [len(pred.split()) for pred in decodedPreds]
        labelLens = [len(label.split()) for label in decodedLabels]
        
        result = {
            "bleu": bleuScore["bleu"],
            "avg_pred_length": np.mean(predLens),
            "avg_label_length": np.mean(labelLens),
        }
        
        # 显示一些样例
        print("\n=== 评估样例 ===")
        for i in range(min(3, len(decodedPreds))):
            print(f"预测 {i+1}: {decodedPreds[i]}")
            print(f"标签 {i+1}: {decodedLabels[i]}")
            print("-" * 50)
        
        return result
        
    except Exception as e:
        print(f"评估过程中出现错误: {e}")
        # 返回基本指标
        predLens = [len(str(pred).split()) for pred in predictions.flatten()]
        labelLens = [len(str(label).split()) for label in labels.flatten()]
        
        return {
            "avg_pred_length": np.mean(predLens),
            "avg_label_length": np.mean(labelLens),
        }

print("评估指标函数已定义")

评估指标函数已定义


In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=computeMetrics,
)

trainer.train()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.72 GiB. GPU 0 has a total capacity of 16.00 GiB of which 0 bytes is free. Of the allocated memory 20.41 GiB is allocated by PyTorch, and 4.73 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained("../output_dir")